In [1]:
import pandas as pd
import sqlite3

In [2]:
# funcoes auxiliares
format_means = lambda x: round(float(x),2)

# abrindo conexao com bd
conn = sqlite3.connect('matches.db')
cursor = conn.cursor()

In [3]:
# criando o df a partir da tabela Odds_1x2
query = 'select odd_home,odd_draw,odd_visiting,fk_id_match from Odds_1x2;'
odds = pd.read_sql(query, conn)

# realizando o calculo da media das odds
odds_avg = odds.groupby(['fk_id_match']).mean()
odds_avg['id_match'] = odds_avg.index

# formatando o dataframe
col_names = {'odd_home':'avg_odds_h',
             'odd_draw': 'avg_odds_d',
             'odd_visiting': 'avg_odds_a'}

odds_avg = odds_avg.rename(columns=col_names)
odds_avg['avg_odds_h'] = odds_avg['avg_odds_h'].apply(format_means)
odds_avg['avg_odds_d'] = odds_avg['avg_odds_d'].apply(format_means)
odds_avg['avg_odds_a'] = odds_avg['avg_odds_a'].apply(format_means)
odds_avg = pd.DataFrame(data=odds_avg, index=[x for x in range(odds_avg.shape[0])])
odds_avg = odds_avg.dropna()
odds_avg['id_match'] = odds_avg['id_match'].astype(int) 
odds_avg.head()

,avg_odds_h,avg_odds_d,avg_odds_a,id_match
1,1.66,4.17,4.92,1
2,6.94,4.81,1.44,2
3,2.48,3.40,2.88,3
4,1.54,4.65,5.54,4
5,1.78,4.17,4.14,5


In [4]:
# criando df a partir da tabela Matches
query = '''select 
                id_match,
                case
                    when goal_home_team > goal_visiting_team then 'H'
                    when goal_home_team < goal_visiting_team then 'A'
                    else 'D'
                end as 'result'
            from Matches;'''
results = pd.read_sql(query, conn)
results.head()

,id_match,result
0,1,H
1,2,H
2,3,A
3,4,H
4,5,A


In [5]:
# juntando os dois dataframes
odds_avg = odds_avg.join(results, on=('id_match'), how='inner',lsuffix='_')

# removendo colunas que nao sao mais necessarias
odds_avg = odds_avg.drop('id_match_',axis=1)
odds_avg = odds_avg.drop('id_match',axis=1)

# fechando conexao com o bd
cursor.close()
conn.close()

odds_avg.head()

,avg_odds_h,avg_odds_d,avg_odds_a,result
1,1.66,4.17,4.92,H
2,6.94,4.81,1.44,A
3,2.48,3.40,2.88,H
4,1.54,4.65,5.54,A
5,1.78,4.17,4.14,H


### Naive Bayes

In [6]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split

# prepara o dataset
y = odds_avg.result
X = odds_avg.drop('result',axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,
                                                    random_state=42)
gnb = GaussianNB()
gnb.fit(X_train, y_train)
gnb.score(X_test, y_test)

0.4559386973180077

### Suport vector machine

In [7]:
from sklearn import svm

clf = svm.SVC()
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.4612437371058061